In [1]:
# 基于空间索引 给时序数据area打标
import findspark
findspark.init('/opt/spark')
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("dwd_to_dwm") \
    .enableHiveSupport() \
    .getOrCreate()

spark.sql("set hive.exec.dynamici.partition=true;")
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict;")


DataFrame[key: string, value: string]

In [2]:
# create database time_series
spark.sql("DROP TABLE IF EXISTS dwm_time_series;")

sql = '''
CREATE TABLE IF NOT EXISTS dwm_time_series
         (id bigint,
          time TIMESTAMP,
          lon float,
          lat float)
PARTITIONED BY(area int)
STORED AS PARQUET;
'''

# create database job
spark.sql("DROP TABLE IF EXISTS dwm_job;")
sql = """

CREATE TABLE IF NOT EXISTS dwm_job
(id bigint,start_time TIMESTAMP,end_time TIMESTAMP)
PARTITIONED BY(yearMonth string)
STORED AS PARQUET;

"""
spark.sql(sql)

# 创建输出表
#CREATE TABLE output(id SERIAL PRIMARY KEY,job_id int4);

print("初始化 dwm_time_series，dwm_job成功")

初始化 dwm_time_series，dwm_job成功


In [3]:
# 加载job 表=> 

sql = '''
INSERT INTO dwm_job partition(yearMonth)
SELECT
 id as id,
 start_time as start_time,
 end_time as end_time,
 yearMonth as yearMonth
 FROM dwd_job;
'''
result = spark.sql(sql)

# 查看一条数据
result = spark.sql("SELECT * FROM dwm_job LIMIT 1")
result.show()


+-----+-------------------+-------------------+---------+
|   id|         start_time|           end_time|yearMonth|
+-----+-------------------+-------------------+---------+
|10208|2025-03-01 00:00:00|2025-03-01 01:00:00|   202503|
+-----+-------------------+-------------------+---------+



In [5]:
# 利用空间索引给数据打标
from sklearn.ensemble import RandomForestRegressor
from rasterio import features
import pandas as pd

# 创建 R 树回归器
r_tree = RandomForestRegressor(n_estimators=100)

sql = '''
SELECT
 id as id,
 time as time,
 lon as lon,
 lat as lat
 FROM dwd_time_series;
'''
print(sql)
result = spark.sql(sql)
result.show()
lon_list = result.select("lon").rdd.flatMap(lambda x: x).collect()
lat_list = result.select("lat").rdd.flatMap(lambda x: x).collect()
data = {'latitude': lat_list, 'longitude': lon_list}
df = pd.DataFrame(data)

# 将经纬度转换为光栅数据
raster = features.rasterize(
    points=df[['latitude', 'longitude']],
    out_shape=(100, 100),
    transform=featuresAffine(
        translate=(-180, 90)
    )
)

# 使用 R 树进行区域划分
r_tree.fit(raster, df['region_id'])

# 对新的经纬度点进行区域划分
new_point = [30, 45]
area = r_tree.predict([[new_point[0], new_point[1]]])[0]
df.write.saveAsTable(dwm_time_series, mode='append', partitionBy=['pt_day'])


SELECT
 id as id,
 time as time,
 lon as lon,
 lat as lat
 FROM dwd_time_series;

+--------+-------------------+---------+---------+
|      id|               time|      lon|      lat|
+--------+-------------------+---------+---------+
|85684740|2026-09-18 10:35:05| 45.27193| 46.27193|
|84348593|2026-09-02 22:49:30|45.267525|46.267525|
|85684741|2026-09-18 10:35:10| 45.27193| 46.27193|
|84348594|2026-09-02 22:49:35|45.267525|46.267525|
|85684742|2026-09-18 10:35:15| 45.27193| 46.27193|
|84348595|2026-09-02 22:49:40|45.267525|46.267525|
|85684743|2026-09-18 10:35:20| 45.27193| 46.27193|
|84348596|2026-09-02 22:49:45|45.267525|46.267525|
|85684744|2026-09-18 10:35:25| 45.27193| 46.27193|
|84348597|2026-09-02 22:49:50|45.267525|46.267525|
|85684745|2026-09-18 10:35:30| 45.27193| 46.27193|
|84348598|2026-09-02 22:49:55|45.267525|46.267525|
|85684746|2026-09-18 10:35:35|45.271935|46.271935|
|84348599|2026-09-02 22:50:00|45.267525|46.267525|
|85684747|2026-09-18 10:35:40|45.271935|46.271935|

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 8.0 failed 4 times, most recent failure: Lost task 2.3 in stage 8.0 (TID 383) (worker1 executor 2): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/hadoop-jupyter/nm-local-dir/usercache/jupyter/appcache/application_1709640126555_0017/container_1709640126555_0017_01_000003/pyspark.zip/pyspark/worker.py", line 540, in main
    raise RuntimeError(
RuntimeError: Python in worker has different version 3.10 than that in driver 3.8, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:559)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:765